# **Problema do Caixeiro Viajante:**
**Definição:** Encontrar o caminho mais curto, que visite todas as cidades somente uma vez e retorne para a origem.
*   Solução 01: Força Bruta, testa todas as opções possivéis, para achar a melhor resposta, porém o custo computacional aumenta exponencialmente com o aumento do número de cidades.
*   Solução 02: Grasp, testa um número reduzido de opções, não garante a melhor resposta, mas uma boa resposta, o custo computacional é drasticamente reduzido.

## **Código:**

In [ ]:
import random as r

class CaixeiroViajante:
    def __init__(self, cidades:list, ligacoes:dict, distancia:dict):
        self.cidades = cidades
        self.ligacaoEntreCidades = ligacoes
        self.distanciaEntreCidades = distancia
        self.interacoes = 0

    def combinacoesPossiveis(self, cidades:list):
        if len(cidades) <= 1:
            return [cidades]

        combinacoes = []

        for i in range(len(cidades)):
            cidadeAtual = cidades[i]
            cidadeRestantes = cidades[:i] + cidades[i+1:]

            outrasCidades = self.combinacoesPossiveis(cidadeRestantes)

            for cidade in outrasCidades:
                combinacoes.append([cidadeAtual] + cidade)

                self.interacoes += 1

        return combinacoes

    def retornoAoInicio(self, combinacoes):
      for combinacao in combinacoes:
        combinacao.append(combinacao[0])

        self.interacoes += 1

      return combinacoes

    def buscarElemento(self, lista:list, elemento):
        for i in range(len(lista)):

            self.interacoes += 1

            if lista[i] == elemento:
                return i

    def getDistanciaEntreCidades(self, cidadeA, cidadeB):
        ligacoes = self.ligacaoEntreCidades[cidadeA]
        localizacao = self.buscarElemento(ligacoes, cidadeB)
        return self.distanciaEntreCidades[cidadeA][localizacao]

    def calcularDistancias(self, rotas:list):
        distancias = []
        for rota in rotas:
            distancia = 0
            for cidade in range(len(rota)-1):
                cidadeAtual = rota[cidade]
                proximaCidade = rota[cidade+1]
                distancia += self.getDistanciaEntreCidades(cidadeAtual, proximaCidade)

                self.interacoes += 1

            distancias.append(distancia)

        return distancias

    def menorDistanciaGeral(self, distancias):
        menorDistancia = distancias[0]
        rota = None

        for i in range(len(distancias)):

            self.interacoes += 1

            if distancias[i] <= menorDistancia:
                menorDistancia = distancias[i]
                rota = i

        return rota

    def cortarLista(self, rotas, alvo):
        if len(rotas) > 0:
            pontos = []
            for i in range(len(rotas)):
                if rotas[i][0] == alvo:
                    pontos.append(i)

                    self.interacoes += 1

            return pontos[0], pontos[len(pontos)-1]

    def menorDistanciaSeletiva(self, distancias, rotas, opcao):
        menorDistancia = distancias[0]
        rota = None

        pontoA, pontoB = self.cortarLista(rotas, opcao)

        for i in range(pontoA, pontoB):

            self.interacoes += 1

            if distancias[i] <= menorDistancia:
                menorDistancia = distancias[i]
                rota = i

        return rota


    def forcaBruta(self, opcao):
        self.interacoes = 0
        rotas = self.combinacoesPossiveis(self.cidades)
        rotas = self.retornoAoInicio(rotas)
        distancias = self.calcularDistancias(rotas)

        if opcao == "":
            rota = self.menorDistanciaGeral(distancias)
        elif opcao in self.cidades:
            rota = self.menorDistanciaSeletiva(distancias, rotas, opcao)

        return rotas[rota], distancias[rota], self.interacoes

    def cidadeMaisProxima(self, cidade, cidadesVisitadas):
        distancias = self.distanciaEntreCidades[cidade]
        ligacoes = self.ligacaoEntreCidades[cidade]
        menorDistancia = distancias[0]*500
        indiceDaCidade = None

        for i in range(len(distancias)):
            self.interacoes += 1
            if distancias[i] <= menorDistancia and ligacoes[i] not in cidadesVisitadas:
                menorDistancia = distancias[i]
                indiceDaCidade = i

        return ligacoes[indiceDaCidade]

    def solucaoInicial(self, cidade):
        rota = []
        rota.append(cidade)

        for i in range(len(self.cidades)-1):
            self.interacoes += 1
            cidadeAtual = rota[len(rota)-1]
            rota.append(self.cidadeMaisProxima(cidadeAtual, rota))

        return rota

    def trocarElementos(self, a, b, lista):
        listaAlterada = [None]*len(lista)
        for i in range(len(lista)):
            self.interacoes += 1
            if i != a and i != b:
                listaAlterada[i] = lista[i]
            elif i == a:
                listaAlterada[i+1] = lista[i]
            elif i == b:
                listaAlterada[i-1] = lista[i]

        return listaAlterada

    def calcularDistanciaPercorrida(self, rota):
        distancia = 0
        for cidade in range(len(rota)-1):
            cidadeAtual = rota[cidade]
            proximaCidade = rota[cidade+1]
            distancia += self.getDistanciaEntreCidades(cidadeAtual, proximaCidade)

            self.interacoes += 1

            self.interacoes += 1

        return distancia

    def buscaLocal(self, rota, numDeBuscas):
        for num in range(numDeBuscas):
            for i in range(len(rota)-2):
                self.interacoes += 1
                rotaAlternativa = self.trocarElementos(i+1, i+2, rota)

                if self.calcularDistanciaPercorrida(rotaAlternativa) <= self.calcularDistanciaPercorrida(rota):
                    rota = rotaAlternativa

        rota.append(rota[0])
        return rota

    def removerElemento(self, lista, elemento):
        listaAlterada = [None]*(len(lista)-1)
        alteracao = 0
        for i in range(len(lista)):
            self.interacoes += 1
            if lista[i] != elemento:
                listaAlterada[i-alteracao] = lista[i]
            else:
                alteracao += 1

        return listaAlterada

    def grasp(self, numDeBuscas:int, numDeRepeticoes:int, alvo:str):
        if numDeRepeticoes > len(self.cidades):
            print("O número de buscas não pode ser maior que o número de cidades")
        else:
            self.interacoes = 0
            rotas = []
            cidadesValidas = self.cidades

            if alvo == "":
                for i in range(numDeRepeticoes):
                    cidadeEscolhida = r.choice(cidadesValidas)
                    cidadesValidas = self.removerElemento(cidadesValidas, cidadeEscolhida)

                    rotaInicial = self.solucaoInicial(cidadeEscolhida)
                    rotaMediana = self.buscaLocal(rotaInicial, numDeBuscas)
                    rotas.append(rotaMediana)

                    self.interacoes += 1

                distancias = self.calcularDistancias(rotas)
            else:
                cidadeEscolhida = alvo
                cidadesValidas = self.removerElemento(cidadesValidas, cidadeEscolhida)

                rotaInicial = self.solucaoInicial(cidadeEscolhida)
                rotaMediana = self.buscaLocal(rotaInicial, numDeBuscas)
                rotas.append(rotaMediana)

                self.interacoes += 1

                distancias = self.calcularDistancias(rotas)


            menorDistancia = distancias[0]
            indice = 0

            for i in range(len(distancias)):
                if distancias[i] <= menorDistancia:
                    menorDistancia = distancias[i]
                    indice = i

                    self.interacoes += 1

            return rotas[indice], distancias[indice], self.interacoes

# **Explicação dos Métodos:**
## Força Bruta:
Simples ele testa todas as opções possiveis e retorna a melhor.

## Grasp:
Como Funciona:
1.   Solução Inicial: escolha deterministica ou randomica da rota inicial.
2.   Busca Local: mudanças de posições para ver se o custo diminui.
3.   Atualiação da melhor solução: caso o custo diminua, a rota original é substituida.
4.   Criterio de Parada: número de vezes que a busca local vai se repetir.
5.   Repetição do Processo: número de vezes que tudo vai se repetir




# **Comparações:**
A comparação vai ser feita para 3, 6 e 9 cidades.

## **Declarando as váriaveis:**

In [ ]:
cidades03 = ["A", "B", "C"]
ligacaoEntreCidades03 = {
    "A": ["B", "C"],
    "B": ["A", "C"],
    "C": ["A", "B"]
}
distanciaEntreCidades03 = {
    "A": [7, 15],
    "B": [7, 7],
    "C": [15, 7]
}



cidades06 = ["A", "B", "C", "D", "E", "F"]
ligacaoEntreCidades06 = {
    "A": ["B", "C", "D", "E", "F"],
    "B": ["A", "C", "D", "E", "F"],
    "C": ["A", "B", "D", "E", "F"],
    "D": ["A", "B", "C", "E", "F"],
    "E": ["A", "B", "C", "D", "F"],
    "F": ["A", "B", "C", "D", "E"]
}
distanciaEntreCidades06 = {
    "A": [7, 15, 10, 30, 75],
    "B": [7, 7, 13, 18, 65],
    "C": [15, 7, 20, 25, 89],
    "D": [10, 13, 20, 20, 45],
    "E": [30, 18, 25, 20, 90],
    "F": [75, 65, 89, 45, 90]
}



cidades09 = ["A", "B", "C", "D", "E", "F", "G", "H", "I"]
ligacaoEntreCidades09 = {
    "A": ["B", "C", "D", "E", "F", "G", "H", "I"],
    "B": ["A", "C", "D", "E", "F", "G", "H", "I"],
    "C": ["A", "B", "D", "E", "F", "G", "H", "I"],
    "D": ["A", "B", "C", "E", "F", "G", "H", "I"],
    "E": ["A", "B", "C", "D", "F", "G", "H", "I"],
    "F": ["A", "B", "C", "D", "E", "G", "H", "I"],
    "G": ["A", "B", "C", "D", "E", "F", "H", "I"],
    "H": ["A", "B", "C", "D", "E", "F", "G", "I"],
    "I": ["A", "B", "C", "D", "E", "F", "G", "H"]
}
distanciaEntreCidades09 = {
    "A": [7, 15, 10, 30, 75, 25, 54, 32],
    "B": [7, 7, 13, 18, 65, 50, 60, 70],
    "C": [15, 7, 20, 25, 89, 45, 92, 43],
    "D": [10, 13, 20, 20, 45, 70, 59, 78],
    "E": [30, 18, 25, 20, 90, 23, 47, 67],
    "F": [75, 65, 89, 45, 90, 11, 87, 12],
    "G": [25, 50, 45, 70, 23, 11, 48, 76],
    "H": [54, 60, 92, 59, 47, 87, 48, 99],
    "I": [32, 70, 43, 78, 67, 12, 76, 99]
}



cidades12 = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"]
ligacaoEntreCidades12 = {
    "A": ["B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"],
    "B": ["A", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"],
    "C": ["A", "B", "D", "E", "F", "G", "H", "I", "J", "K", "L"],
    "D": ["A", "B", "C", "E", "F", "G", "H", "I", "J", "K", "L"],
    "E": ["A", "B", "C", "D", "F", "G", "H", "I", "J", "K", "L"],
    "F": ["A", "B", "C", "D", "E", "G", "H", "I", "J", "K", "L"],
    "G": ["A", "B", "C", "D", "E", "F", "H", "I", "J", "K", "L"],
    "H": ["A", "B", "C", "D", "E", "F", "G", "I", "J", "K", "L"],
    "I": ["A", "B", "C", "D", "E", "F", "G", "H", "J", "K", "L"],
    "J": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L"],
    "K": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "L"],
    "L": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]
}
distanciaEntreCidades12 = {
    "A": [7, 15, 10, 30, 75, 25, 54, 32, 33, 44, 66],
    "B": [7, 7, 13, 18, 65, 50, 60, 70, 12, 44, 55],
    "C": [15, 7, 20, 25, 89, 45, 92, 43, 72, 33, 34],
    "D": [10, 13, 20, 20, 45, 70, 59, 78, 16, 58, 71],
    "E": [30, 18, 25, 20, 90, 23, 47, 67, 78, 9, 46],
    "F": [75, 65, 89, 45, 90, 11, 87, 12, 73, 95, 71],
    "G": [25, 50, 45, 70, 23, 11, 48, 76, 61, 29, 86],
    "H": [54, 60, 92, 59, 47, 87, 48, 99, 64, 41, 42],
    "I": [32, 70, 43, 78, 67, 12, 76, 99, 88, 90, 83],
    "J": [33, 12, 72, 16, 78, 73, 61, 64, 88, 66, 92],
    "K": [44, 44, 33, 58, 9, 95, 29, 41, 90, 66, 36],
    "L": [66, 55, 34, 71, 46, 71, 86, 42, 83, 92, 36]
}

## **Com três cidades (A, B, C):**
### Força Bruta:
Rápido e retornou a melhor opção.
### Grasp:
Alcançou um bom resultado, mas demorou mais.

In [ ]:
cidadeDeLargada = ""
cidades = cidades03
ligacaoEntreCidades = ligacaoEntreCidades03
distanciaEntreCidades = distanciaEntreCidades03

caixeiro = CaixeiroViajante(cidades, ligacaoEntreCidades, distanciaEntreCidades)

rota, distancia, interacoes = caixeiro.forcaBruta(cidadeDeLargada)
print(f"\nForça Bruta:\nMelhor rota: {rota}\nDistancia percorrida: {distancia}\nInteracoes: {interacoes}")

rota, distancia, interacoes = caixeiro.grasp(2, 3, cidadeDeLargada) # 2 = número de repetições do método de busca local; 3 = número de largadas testadas
print(f"\nGrasp:\nMelhor rota: {rota}\nDistancia percorrida: {distancia}\nInteracoes: {interacoes}")


Força Bruta:
Melhor rota: ['C', 'B', 'A', 'C']
Distancia percorrida: 29
Interacoes: 69

Grasp:
Melhor rota: ['B', 'C', 'A', 'B']
Distancia percorrida: 29
Interacoes: 160


## **Com seis cidades (A, B, C, D, E, F):**
### Força Bruta:
Demorou, mas retornou a melhor opção.
### Grasp:
Alcançou um bom resultado muito rápido.

In [ ]:
cidadeDeLargada = ""
cidades = cidades06
ligacaoEntreCidades = ligacaoEntreCidades06
distanciaEntreCidades = distanciaEntreCidades06

caixeiro = CaixeiroViajante(cidades, ligacaoEntreCidades, distanciaEntreCidades)

rota, distancia, interacoes = caixeiro.forcaBruta(cidadeDeLargada)
print(f"\nForça Bruta:\nMelhor rota: {rota}\nDistancia percorrida: {distancia}\nInteracoes: {interacoes}")

rota, distancia, interacoes = caixeiro.grasp(2, 6, cidadeDeLargada) # 2 = número de repetições do método de busca local; 6 = número de largadas testadas
print(f"\nGrasp:\nMelhor rota: {rota}\nDistancia percorrida: {distancia}\nInteracoes: {interacoes}")


Força Bruta:
Melhor rota: ['F', 'D', 'E', 'C', 'A', 'B', 'F']
Distancia percorrida: 177
Interacoes: 22320

Grasp:
Melhor rota: ['B', 'A', 'C', 'E', 'D', 'F', 'B']
Distancia percorrida: 177
Interacoes: 3029


## **Com nove cidades (A, B, C, D, E, F, G, H, I):**
### Força Bruta:
Demorou bastante, mas retornou a melhor opção.
### Grasp:
Alcançou um bom resultado rapidamente.

In [ ]:
cidadeDeLargada = ""
cidades = cidades09
ligacaoEntreCidades = ligacaoEntreCidades09
distanciaEntreCidades = distanciaEntreCidades09

caixeiro = CaixeiroViajante(cidades, ligacaoEntreCidades, distanciaEntreCidades)

rota, distancia, interacoes = caixeiro.forcaBruta(cidadeDeLargada)
print(f"\nForça Bruta:\nMelhor rota: {rota}\nDistancia percorrida: {distancia}\nInteracoes: {interacoes}")

rota, distancia, interacoes = caixeiro.grasp(2, 9, cidadeDeLargada) # 2 = número de repetições do método de busca local; 9 = número de largadas testadas
print(f"\nGrasp:\nMelhor rota: {rota}\nDistancia percorrida: {distancia}\nInteracoes: {interacoes}")


Força Bruta:
Melhor rota: ['I', 'F', 'G', 'H', 'E', 'D', 'C', 'B', 'A', 'I']
Distancia percorrida: 204
Interacoes: 21591360

Grasp:
Melhor rota: ['G', 'F', 'I', 'A', 'B', 'C', 'D', 'E', 'H', 'G']
Distancia percorrida: 204
Interacoes: 15347


## **Com doze cidades (A, B, C, D, E, F, G, H, I, J, K, L):**
### Força Bruta:
Não ressolveu porque crashou o COLAB, uso extremo da mémoria ram.
### Grasp:
Alcançou um resultado em um bom tempo.

In [ ]:
cidadeDeLargada = ""
cidades = cidades12
ligacaoEntreCidades = ligacaoEntreCidades12
distanciaEntreCidades = distanciaEntreCidades12

caixeiro = CaixeiroViajante(cidades, ligacaoEntreCidades, distanciaEntreCidades)

#rota, distancia, interacoes = caixeiro.forcaBruta(cidadeDeLargada)
#print(f"\nForça Bruta:\nMelhor rota: {rota}\nDistancia percorrida: {distancia}\nInteracoes: {interacoes}")

rota, distancia, interacoes = caixeiro.grasp(2, 12, cidadeDeLargada) # 2 = número de repetições do método de busca local; 12 = número de largadas testadas
print(f"\nGrasp:\nMelhor rota: {rota}\nDistancia percorrida: {distancia}\nInteracoes: {interacoes}")


Grasp:
Melhor rota: ['E', 'K', 'G', 'F', 'I', 'A', 'B', 'C', 'D', 'J', 'H', 'L', 'E']
Distancia percorrida: 295
Interacoes: 47931
